## Script to move raw MACS files based on extracted footprints files 
**author:** Ingmar Nitze, Tabea Rettelbach, Simon Schäffler

**contact:** ingmar.nitze@awi.de

**version date:** 2022-02-24

**repository and other tools** https://github.com/awi-response/MACS_tools

## Contents
1. Setup folder structure
2. convert MACS files to TIFF with *mipps* including devignetting
3. Rescale image values **Optional**
4. Crop corners of images **Optional**
5. Prepare nav files for Pix4d

## Settings 

* prefer full/absolute paths
* Create processing template automatically

In [ ]:
# Set project directory here, where you want to process your data
PROJECT_DIR = r''
#PROJECT_DIR = r'D:\pix4d_Processing\ThawTrendAir_2019\Image_Test_CODEscaleHi' # SET Project output

# Set raw data dir here for the speicific image acquisition project
path_infiles = r''
#path_infiles = r'N:\response\Restricted_Airborne\MACs\Alaska\ThawTrend-Air_2019\raw_data\20190727-235440_15L_Ketik_fire_flight_plan_v3'

# determine which sensors to include in processing (possible options: 'left', 'right', 'nir')
sensors = ['left', 'right', 'nir']

# Set CROP CORNER if 
CROP_CORNER = 0 # SET to 1 if you want to crop corners (set to NoData)
DISK_SIZE = 5200 # Cropping diameter, the larger the fewer no data

# SET SCALING 
SCALING = 1
SCALE_LOW = False # Set to True to use calculated lower boundary - skews NDVI
SCALE_HIGH = True # Set to True to use calculated upper boundary

### Imports 

In [ ]:
import geopandas as gpd
import shutil
import os
import glob
import pandas as pd
from IPython.display import clear_output
import sys
import numpy as np
import tqdm
import zipfile
from pathlib import Path
from joblib import delayed, Parallel, wrap_non_picklable_objects
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from processing_utils import *

In [ ]:
pwd = Path(os.getcwd())

#### Fixed Settings
* These Settings can be kept

In [ ]:
CODE_DIR = pwd
MIPPS_DIR = r'C:\Program Files\DLR MACS-Box\bin'
MIPPS_BIN = r'..\tools\MACS\mipps.exe'
EXIF_PATH = Path(CODE_DIR / Path(r'exiftool\exiftool.exe'))
mipps_script_dir = Path('mipps_scripts')

In [ ]:
mipps_script_nir = '33552_all_taps_2018-09-26_12-58-15_modelbased.mipps'
mipps_script_right = '33576_all_taps_2018-09-26_13-13-43_modelbased.mipps'
mipps_script_left = '33577_all_taps_2018-09-26_13-21-24_modelbased.mipps'

mipps_script_nir = pwd / mipps_script_dir / mipps_script_nir
mipps_script_right = pwd / mipps_script_dir / mipps_script_right
mipps_script_left = pwd / mipps_script_dir / mipps_script_left

In [ ]:
DATA_DIR = Path(PROJECT_DIR) / '01_rawdata' / 'tif'
OUTDIR = {'right': DATA_DIR / Path('33576_Right'),
          'left':DATA_DIR / Path('33577_Left'),
          'nir':DATA_DIR / Path('33552_NIR')}
tag = {'right':'MACS_RGB_Right_33576',
       'left':'MACS_RGB_Left_33577',
       'nir':'MACS_NIR_33552'}

In [ ]:
# Path of filtered footprints file (.shp file)
path_footprints = os.path.join(PROJECT_DIR, '02_studysites','footprints.shp')
outdir = os.path.join(PROJECT_DIR, '01_rawdata','tif')

#### Prepare processing dir 
* check if exists

In [ ]:
zippath = os.path.join(CODE_DIR, 'processing_folder_structure_template.zip')
nav_script_path = os.path.join(CODE_DIR, 'pix4dnav.py')

In [ ]:
with zipfile.ZipFile(zippath, 'r') as zip_ref:
    zip_ref.extractall(PROJECT_DIR)
shutil.copy(nav_script_path, outdir)

### Manual Selection of footprints 

Now select footprints and export selection as ***footprints.shp*** to ***02_footprints*** in your working directory

#### Load filtered footprints file 

In [ ]:
df_final = prepare_df_for_mipps(path_footprints, path_infiles)

#### Workaround to deal with spaces in path" 

In [ ]:
df_final['full_path'] = df_final.apply(lambda x: f'"{x.full_path}"', axis=1)

In [ ]:
print("Total number of images:", len(df_final))
print("NIR images:", (df_final['Looking'] == 'center').sum())
print("RGB right images:", (df_final['Looking'] == 'right').sum())
print("RGB left images:", (df_final['Looking'] == 'left').sum())

#### Run Process 

In [ ]:
os.chdir(MIPPS_DIR)

In [ ]:
max_roll = 3 # Select maximum roll angle to avoid image issues - SET in main settings part?
chunksize = 20 # this is a mipps-script thing

In [ ]:
# this is relevant for NIR only
if 'nir' in sensors:
    looking = 'center'
    q = (np.abs(df_final['Roll[deg]']) < max_roll) & (df_final['Looking'] == looking)
    df_nir = df_final[q]
    print(len(df_nir))
    for df in tqdm.tqdm_notebook(np.array_split(df_nir, len(df_nir) // chunksize)):
        outlist = ' '.join(df['full_path'].values[:])
        s = f'{MIPPS_BIN} -c={mipps_script_nir} -o={outdir} -j=4 {outlist}'
        os.system(s)
        #print(s)

In [ ]:
# this is RGB
if 'right' in sensors:
    looking = 'right'
    q = (np.abs(df_final['Roll[deg]']) < max_roll) & (df_final['Looking'] == looking)
    df_right = df_final[q]
    for df in tqdm.tqdm_notebook(np.array_split(df_right, len(df_right) // chunksize)):
        outlist = ' '.join(df['full_path'].values[:])
        s = f'{MIPPS_BIN} -c={mipps_script_right} -o={outdir} -j=4 {outlist}'
        os.system(s)

In [ ]:
if 'left' in sensors:
    looking = 'left'
    q = (np.abs(df_final['Roll[deg]']) < max_roll) & (df_final['Looking'] == looking)
    df_left = df_final[q]
    for df in tqdm.tqdm_notebook(np.array_split(df_left, len(df_left) // chunksize)):
        outlist = ' '.join(df['full_path'].values[:])
        s = f'{MIPPS_BIN} -c={mipps_script_left} -o={outdir} -j=4 {outlist}'
        os.system(s)

### Rescale image values 

#### Image Statistics 

In [ ]:
if SCALING:
    %time df_stats = get_image_stats_multi(OUTDIR, sensors, nth_images=10, quiet=True)
    #absolute
    if SCALE_LOW:
        scale_lower = int(df_stats['min'].mean().round())
    else:
        scale_lower = 1
    if SCALE_HIGH:
        scale_upper = int(df_stats['max'].mean().round())
    else:
        scale_upper = 2*16-1
    print(f'Mean of minimums: {scale_lower}')
    print(f'Mean of maximums: {scale_upper}')

#### Run scaling
* minimum default to 1
* consistency for final index calculation

In [ ]:
if SCALING:
    n_jobs = 20
    for sensor in sensors:
        print(f'Processing {sensor}')
        #shutter_factor
        images = list(OUTDIR[sensor].glob('*.tif'))[:]
        if sensor in ['right', 'left']:
            shutter_factor = get_shutter_factor(OUTDIR, sensors)
            print(f'RGB to NIR factor = {shutter_factor}')
        else:
            shutter_factor = 1
        
        %time _ = Parallel(n_jobs=n_jobs)(delayed(write_new_values)(image, scale_lower, scale_upper, shutter_factor=shutter_factor, tag=True) for image in tqdm.tqdm_notebook(images[:]))

#### Crop Corners of images 

In [ ]:
if CROP_CORNER:
    #mask = make_mask((3232, 4864), disksize=DISK_SIZE)
    for sensor in sensors[:]:
        mask = make_mask((3232, 4864), disksize=DISK_SIZE)
        images = list(OUTDIR[sensor].glob('*'))
        if sensor != 'nir':
            mask = np.r_[[mask]*3]
        #%time _ = [mask_and_tag(image, mask, tag=None) for image in tqdm.tqdm_notebook(images)]
        %time _ = Parallel(n_jobs=4)(delayed(mask_and_tag)(image, mask, tag=None) for image in tqdm.tqdm_notebook(images))

#### Write exif information into all images 

In [ ]:
for sensor in tqdm.tqdm_notebook(sensors):
    print(sensor)
    %time write_exif(OUTDIR[sensor], tag[sensor], EXIF_PATH)

#### Nav

In [ ]:
navfile = list(Path(path_infiles).glob('*nav.txt'))[0]

In [ ]:
shutil.copy(navfile, OUTDIR['nir'].parent / 'nav.txt')

In [ ]:
os.chdir(OUTDIR['nir'].parent)
os.system('python pix4dnav.py')

### Create 4 band CIR mosaic

In [ ]:
os.chdir(Path(PROJECT_DIR) / '04_pix4d' / Path(PROJECT_DIR).name / '3_dsm_ortho' / '2_mosaic')

In [ ]:
rgbfile = list(Path('.').glob('*group1.tif'))[0]
nirfile = list(Path('.').glob('*nir.tif'))[0]
outmosaic = '_'.join(rgbfile.name.split('_')[:-3])+'_mosaic.tif'

#### Export single bands and merge afterwards 

In [ ]:
for band in [1,2,3]:
    s = f'gdal_translate -b {band} -co COMPRESS=DEFLATE {rgbfile} rgb_{band}.tif'
    os.system(s)

for band in [1]:
    s = f'gdal_translate -b {band} -co COMPRESS=DEFLATE {nirfile} nir_{band}.tif'
    os.system(s)

s = f'gdalbuildvrt -separate 4band.vrt rgb_3.tif rgb_2.tif rgb_1.tif nir_1.tif'
os.system(s)

s = f'gdal_translate -a_nodata 0 -co COMPRESS=DEFLATE 4band.vrt {outmosaic}'
os.system(s)


for file in ['rgb_1.tif', 'rgb_2.tif', 'rgb_3.tif', 'nir_1.tif', '4band.vrt']:
    os.remove(file)


### Mask 
* set nodata
* set mask to valid in all bands only

In [ ]:
with rasterio.open(outmosaic, 'r+') as src:
    src.profile['nodata'] = 0
    data = src.read()
    newmask = ~(data == 0).any(axis=0)
    newmask_write = np.r_[src.count * [newmask]]
    data_masked = data * newmask_write
    src.write(data_masked)